# Generators

### Definition:

* A generator is a function that returns a *generator iterator*.

Just like an iterator, a generator iterator performs some operation when `next()` is called on it. This is what happens inherently when one used a for loop.

### However,

There is one **big** difference. Where iterators tend to be used until exhausted, a generator iterator temporarily `yields` control back to the user.

What this means is a generator remains stateful while it *waits* for the user to call it again.

![](http://nvie.com/img/relationships.png)

### Generator vs Function

While a generator is a Python function that returns a generator iterator, there is a big difference between the two.

**Functions** start with the first line and processes until it `return`s (or `prints`, `log`s , etc) something. It is then effectively *reset*. This means that if your were to call that same function again, it would build all of its variables and objects (within its scope) from scratch.

The most precise definition is that a function that stores local variables during operation, then loses them when complete is called a <u>subroutine</u>

**Generators**, however, keep their *state* (their variables and objects) after resolving the `yield` statement. This means that if you call a generator again, it will pick up where it left off. (also known as coroutines).

## Why does this matter?

Often, especially in bioinformatics, we are operating with very large amounts of data. For example, let's say we are working with Hi-C data, which contains an all-vs-all Pearson Correlation matrix.


<center>![](http://web.cmb.usc.edu/people/alber/images/HiCorrector.png)</center>

Say I am doing some weird analysis where I only want to look at all the regions that have a correlation between 
```math
0.8 < corr < 0.95
```....I don't know why, just saying.

To pull out that information, with a <u>traditional iterator</u>, I would have to go through the whole matrix, collect all the values, then return them in some format (`list`, `dict`, `tuple`, etc.) This list could, potentially, be  very **very** large.

<center>With a <u>generator</u>, it could find the first N values, then wait. ![](https://media.giphy.com/media/4KkSbPnZ5Skec/giphy.gif)</center>

<center>then give you the next N values when you call `next()` again. The best part is, it is lazy (just like me). It wont do anything until you tell it to. This makes big data easy to iteratively process.</center>

### An example (Project Euler stuff)

In [134]:
# a utility function
import math
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
import numpy as np


def is_prime(number):
    if number % 2 == 0 and number > 2: 
        return False
    return all(number % i for i in range(3, int(math.sqrt(number)) + 1, 2))

Now, given a list of numbers `nums`, return all of the prime numbers

In [135]:
def get_primes(input_list):
    result_list = []
    for element in input_list:
        if is_prime(element):
            result_list.append(element)

    return result_list

With a regular function and a relatively short list of numbers, this is a perfectly good solution.

In [136]:
%time get_primes(np.arange(1, 1e2))

Wall time: 1 ms


[1.0,
 2.0,
 3.0,
 5.0,
 7.0,
 11.0,
 13.0,
 17.0,
 19.0,
 23.0,
 29.0,
 31.0,
 37.0,
 41.0,
 43.0,
 47.0,
 53.0,
 59.0,
 61.0,
 67.0,
 71.0,
 73.0,
 79.0,
 83.0,
 89.0,
 97.0]

What about when we get bigger? Say the list is infinitely long. So long that it wouldn't fit in memory. Let's do some benchmarking...

In [137]:
%time get_primes(np.arange(1, 1e2))
%time get_primes(np.arange(1, 1e3))
%time get_primes(np.arange(1, 1e4))
%time get_primes(np.arange(1, 1e5))
%time get_primes(np.arange(1, 1e6))
print("Done!")

Wall time: 0 ns
Wall time: 2 ms
Wall time: 35.1 ms
Wall time: 542 ms
Wall time: 12.1 s
Done!


Now, let's try a generator...

In [138]:
def get_primes(number):
    while True:
        if is_prime(number):
            yield number
        number += 1

In [139]:
def pass_primes(stop):
    global primer
    primer = get_primes(1)
    primer.send(None)
    for prime in primer:
        if prime <= stop:
            pass
        else:
            break

In [140]:
%time pass_primes(1e2)
%time pass_primes(1e3)
%time pass_primes(1e4)
%time pass_primes(1e5)
%time pass_primes(1e6)
%time pass_primes(1e7)

Wall time: 0 ns
Wall time: 1 ms
Wall time: 13.1 ms
Wall time: 173 ms
Wall time: 3.28 s
Wall time: 1min 19s


In [142]:
next(primer)

10000103

## Generator Expressions

We already know what a list comprehension is. For example:

In [143]:
list_comp = [i for i in np.arange(1, 1e6) if i % 2 == 0]
list_comp[:5]

[2.0, 4.0, 6.0, 8.0, 10.0]

However, this suffers the same limitation as our `get_primes()` example. It ***eagerly*** computes all the possibilities, *then* it returns a list as an object.

A generator expression gives you the same power, but it does so ***lazily***

In [144]:
gen_comp = (i for i in np.arange(1, 1e6) if i % 2 == 0)
gen_comp

<generator object <genexpr> at 0x000002593C2DDF68>

WAIT, WAIT, WAIT! That isn't the same thing,...right?

In [145]:
for i in range(5):
    print(next(gen_comp))

2.0
4.0
6.0
8.0
10.0


It *is* the same thing in that it gives you the same data. The only difference is that it doesn't give you *all* the data at one time. It only gives you the next data point, thus saving you on memory resources.

# Workshop

In pairs, develop a generalized program that will return the fibonacci sequence between I and J. If that sequence is not N items long, exand the support until it is.

```python
seq = fib_seq(I, J)
len(seq) == N
>>> True
```